In [ ]:
import arcpy as arc

In [ ]:
arc.env.workspace="C:\\Users\\Zeke\\Desktop\\Geospatial_final_workspace"
print(arc.env.workspace)

In [ ]:
# Define shapefile for Census block group data

shapefile_name = "IA_blck_grp_2022.shp"

# Output feature class name
output_fc_name = "IA_blck_grp_2022"

# Copy the shapefile to a feature class
arc.management.CopyFeatures(shapefile_name, output_fc_name)

In [ ]:
#GTFS usually follow simliar template with file organization and often are extracted as txt files
# Use arc tools to create feature class for route and stops

arc.conversion.GTFSShapesToFeatures("C:\\Users\\Zeke\\Downloads\\gtfs\\shapes.txt", 'routes')


arc.transit.GTFSStopsToFeatures("C:\\Users\\Zeke\\Downloads\\gtfs\\stops.txt", 'stops')

In [ ]:
#define the projection of new 'routes' ans 'stops' to match CBG
projection_source = 'IA_blck_grp_2022.shp'

# Layers to be reprojected
routes_layer = "routes"
stops_layer = "stops"

# Define the projection
arcpy.management.DefineProjection(routes_layer, projection_source)
arcpy.management.DefineProjection(stops_layer, projection_source)

In [ ]:
#Select Block groups that median income is below living wage threshold per MIT
arc.management.SelectLayerByAttribute(
    in_layer_or_view="IA_blck_grp_2022",
    selection_type="NEW_SELECTION",
    where_clause="AQP6M001 <= 35786",
    invert_where_clause=None
)

#Make layer from selection
arc.management.CopyFeatures("IA_blck_grp_2022","blk_grp_no_routes")

In [ ]:
#Spatial Join routes and block groups
arc.analysis.SpatialJoin("blk_grp_no_routes","routes", 'blk_grp_wroutes.shp', 'JOIN_ONE_TO_MANY')

In [ ]:
#Generate service areas around each stop that cover a 10 minute walking range
arc.agolservices.GenerateServiceAreas("stops_IC","10", "Minutes")

In [ ]:
#Using Arc Pro import POI data from SafeGraph
with arc.EnvManager(extent='362106.118999999 467997.711200001 375052.946199998 479007.7938 PROJCS["USA_Contiguous_Albers_Equal_Area_Conic",GEOGCS["GCS_North_American_1983",DATUM["D_North_American_1983",SPHEROID["GRS_1980",6378137.0,298.257222101]],PRIMEM["Greenwich",0.0],UNIT["Degree",0.0174532925199433]],PROJECTION["Albers"],PARAMETER["False_Easting",0.0],PARAMETER["False_Northing",0.0],PARAMETER["Central_Meridian",-96.0],PARAMETER["Standard_Parallel_1",29.5],PARAMETER["Standard_Parallel_2",45.5],PARAMETER["Latitude_Of_Origin",37.5],UNIT["Meter",1.0]]'):
    arc.ba.GeneratePointsFromBusinessListings(
        out_feature_class=r"C:\Users\Zeke\Desktop\Geospatial_final_workspace\POI.shp",
        in_search_features="IA_blck_grp_2022",
        search_terms="",
        exact_match="PARTIAL_MATCH",
        match_name_only="MATCH_ALL_FIELDS",
        filters=None,
        max_count=5000,
        business_dataset="US.SafeGraphBUS"
    )

In [ ]:
#If needed use select by attribute to specify data to correct extent
arc.management.SelectLayerByLocation(
    in_layer="POI",
    overlap_type="WITHIN",
    select_features="blk_grp_wroutes",
    search_distance=None,
    selection_type="NEW_SELECTION",
    invert_spatial_relationship="NOT_INVERT"
)

#Create layer from selection
arc.management.CopyFeatures("POI", "IC_POI")

In [ ]:
#Using Labor Stats match adjusted mean income to industry of POI- matching the income dict by the first two digits of NAICS codes

fc_name = "IC_POI"  # Update with your feature class name

# Input dictionary containing NAICS code mappings
income_dict = {
    "23": 65222,
    "31": 26224,
    "32": 71595,
    "33": 26584,
    "42": 71595,
    "44": 26935,
    "45": 26935,
    "48": 57772,
    "49": 57772,
    "51": 40197,
    "52": 63669,
    "53": 77452,
    "54": 39466,
    "55": 40314,
    "56": 63669,
    "61": 61088,
    "62": 61088,
    "71": 63669,
    "72": 64085,
    "81": 64085,
    "92": 26935
}

# Add new field to the feature class
new_field_name = 'Income'  # Name of the new field to be added
field_type = 'TEXT'  # Data type of the new field
field_length = 10  # Length of the new field (adjust as needed)

arc.AddField_management(in_table=fc_name,
                          field_name=new_field_name,
                          field_type=field_type,
                          field_length=field_length)

# Update the new field based on NAICS codes
with arc.da.UpdateCursor(fc_name, ['naics_code', new_field_name]) as cursor:
    for row in cursor:
        naics_code = row[0]
        first_two_digits = naics_code[:2]  # Get the first two digits of NAICS code
        if first_two_digits in income_dict:
            row[1] = income_dict[first_two_digits]
        else:
            row[1] = 'NA'  # Set a default value if NAICS code not found in dictionary
        cursor.updateRow(row)


In [ ]:
#Filter POI data based on places that average livable wage or above 
# Set the income threshold
income_threshold = 35786

where_clause= f"Income_1 >= {income_threshold}"

# Perform attribute query to select points with income above the threshold
arc.management.SelectLayerByAttribute(
    in_layer_or_view="IC_POI",
    selection_type="NEW_SELECTION",
    where_clause=where_clause
)

# Save the selected points to a new feature class
arc.management.CopyFeatures(
    in_features="IC_POI",
    out_feature_class="workplaces"
)

In [ ]:
#Filter workplaces that are within CBG and associated bus routes
arc.management.SelectLayerByLocation("workplaces","WITHIN","blk_grp_wroutes")

#Make into layer
arc.management.CopyFeatures(
    in_features="workplaces",
    out_feature_class="Total_workplaces_in_block_group"
)

In [ ]:
#Filter total work places by the generated "Service Areas" to determine which workplaces are accessible by transit
arc.management.SelectLayerByLocation("Total_workplaces_in_block_group", "WITHIN", "Service Areas")

#Make into layer
arc.management.CopyFeatures(
    in_features="Total_workplaces_in_block_group",
    out_features_class="workplaces_transit_filtered"
)

In [ ]:
cbg_shp="blk_grp_no_routes" #use unjoined block group layer to avoid creating values for 'route' shapes
wta="workplaces_transit_filtered"
wt="Total_workplaces_in_block_group"

# Check if the fields exist in the CBG feature class, if not, create them
field_names = [field.name for field in arcpy.ListFields(cbg_shp)]
if "CWC" not in field_names:
    arc.AddField_management(cbg_shp, "CWC", "LONG")
if "PCW" not in field_names:
    arc.AddField_management(cbg_shp, "PCW", "DOUBLE")
# Loop through each CBG and calculate the number and percentage of transit-accessible workplaces
with arc.da.SearchCursor(cbg_shp, ["GEOID"]) as cursor:
    for row in cursor:
        cbg_oid = row[0]
        print(cbg_oid)
        
        # Define the query expression to select the current CBG based on its ObjectID
        query_expression = "GEOID ='{}'".format(cbg_oid)
        print(query_expression)
        
        # Select the current CBG in the CBG layer
        arc.management.SelectLayerByAttribute(cbg_shp, "NEW_SELECTION", query_expression)

        # Calculate the percentage of transit-accessible workplaces for the current CBG
        arc.management.SelectLayerByLocation(wt, "WITHIN", cbg_shp)
        total_workplaces_count = arc.GetCount_management(wt).getOutput(0)
        print(total_workplaces_count)
        
        # Select the transit-accessible workplaces within the current CBG
        arc.management.SelectLayerByLocation(wta, "WITHIN", cbg_shp)

        # Count the number of selected transit-accessible workplaces
        conducive_workplaces_count = arc.GetCount_management(wta).getOutput(0)
        print(conducive_workplaces_count)
        
        # Calculate the percentage of transit-accessible workplaces for the current CBG
        if int(total_workplaces_count) == 0:
            percent_conducive_workplaces = 0  # Set percentage to 0 if total is zero
        else:
            percent_conducive_workplaces = (int(conducive_workplaces_count) / int(total_workplaces_count)) * 100
            print(percent_conducive_workplaces)

        # Update the CBG feature with the conducive workplaces count and percentage
        with arc.da.UpdateCursor(cbg_shp, ["CWC", "PCW"], query_expression) as update_cursor:
            for update_row in update_cursor:
                update_row[0] = conducive_workplaces_count
                update_row[1] = percent_conducive_workplaces
                update_cursor.updateRow(update_row)

        # Clear the selection
        arc.management.SelectLayerByAttribute(cbg_shp, "CLEAR_SELECTION")


In [ ]:
#Field 'CWC' is the total amount of workplaces in a given CBG that due to meeting livable wage threshold is viewed as economic mobility factor
# Field 'PCW' is a percentage value of accessibilty to CWC in a given CBG via transit